In [ ]:
# Fine tuning Chat models for new GPT 3.5 turbo model
# Perform entity extraction using RecipeNLG dataset

# Step 1: Loading the dataset and filtering down to one domain to fine-tune on

# Step 2: Data Preparation : Preparing data for fine-tuning by creating training and validation examples
# and uploading them to the Files endpoint

# Step 3: Fine-tuning :  Creating our fine-tuned model

# Step 4: Inference : Using fine-tuned model for inference on new inputs

# Train, evaluate and deploy a fine-tuned gpt-3.5-turbo model



In [1]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

OPENAI_API_KEY = os.getenv("824fe43e851f4862af326fa83c3d3cfe")

In [8]:
# Fine-tuning works best when focused on a particular domain.os
# Its important to make sure that dataset is both focused enough for the model to learn, but general enough that unseen examples won't be missed
# To do so, must extract a subset from the RecipesNLG dataset to only contain documents from www.cookbooks.com.

# Read in the dataset to use for the task
# This will be the RecipesNLG dataset, which we've cleaned to only contain documents from www.cookbooks.com
recipe_df = pd.read_csv("data/cookbook_recipes_nlg_10k.csv")
recipe_df.head()


,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
# Data preparation
# When fine tuning with the Chat Completion format, each training example is a simple list of messages
# For example, an entry could look like:
# [{'role':'system',
#   'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},
# ]
# { 'role': 'user',
#    'content': 'Title: No-Bake Nut Cookies\n\Ingredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},
# {'role': 'assistant',
# 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]] '
#
#}

# During the training process this conversation will be split, with the final entry being the completion that the mpd